---
subject: Bases de données
venue: SQL
title: "Présentation de xeus-sqlite"
subtitle: Requêtes SQL et graphiques XVEGA avec Jupyterlite
authors:
  - name: Davy Cottet
    affiliations:
      - Lycée Expérimental
license: CC-BY-NC-4.0
---

Bienvenue dans ce tutoriel interactif ! Ce notebook a pour but de vous introduire aux bases du langage SQL (Structured Query Language) pour interroger des bases de données, ainsi qu'à la création de visualisations de données directement depuis vos requêtes grâce à la fonctionnalité `XVEGA_PLOT`.

Nous utiliserons le noyau **xsqlite** pour Jupyter, qui permet d'exécuter des commandes SQL et des commandes spéciales (appelées "magiques") directement dans les cellules d'un notebook.

## 1. Les bases de SQL : Créer et interroger une base de données

Commençons par créer notre propre base de données. Pour cela, nous utilisons une commande magique propre au noyau `xsqlite`.

### Création d'une base de données

La commande `%CREATE` permet de créer un nouveau fichier de base de données SQLite. Si le fichier existe déjà, cette commande l'ouvrira.

````{admonition} Commandes Magiques
:class: tip
Dans le noyau `xsqlite`, les commandes qui commencent par un `%` ne sont pas du SQL standard. Ce sont des instructions spécifiques au noyau pour gérer la base de données (créer, charger, supprimer, etc.) ou pour générer des visualisations.
````

In [ ]:
%CREATE example_db.db

### Création d'une table

Maintenant que nous avons une base de données, nous avons besoin d'une structure pour stocker nos données : une **table**. Une table est organisée en colonnes (avec un nom et un type de données) et en lignes (les enregistrements).

La commande `CREATE TABLE` permet de définir cette structure. Nous allons créer une table `players` pour stocker des informations sur des personnages de jeu.


In [ ]:
CREATE TABLE players (Name STRING, Class STRING, Level INTEGER, Hitpoints INTEGER)

### Insertion de données

Notre table `players` est vide. Utilisons la commande `INSERT INTO` pour y ajouter des lignes (des personnages).

La syntaxe est la suivante : `INSERT INTO nom_table (colonne1, colonne2) VALUES (valeur1, valeur2);`

In [ ]:
INSERT INTO players (Name, Class, Level, Hitpoints) VALUES ("Martin Splitskull", "Warrior", 3, 40)

In [ ]:
INSERT INTO players (Name, Class, Level, Hitpoints) VALUES ("Sir Wolf", "Cleric", 2, 20)

In [ ]:
INSERT INTO players (Name, Class, Level, Hitpoints) VALUES ("Sylvain, The Grey", "Wizard", 1, 10)

### Sélection de données

La commande la plus courante en SQL est `SELECT`. Elle permet d'extraire des données d'une table.

Pour afficher uniquement les colonnes `Level` et `Hitpoints` de tous les joueurs, nous exécutons la requête suivante :

In [ ]:
SELECT Level, Hitpoints FROM players

### Fonctions d'agrégation

SQL peut également effectuer des calculs sur des ensembles de données. Par exemple, pour connaître la somme des niveaux de tous les joueurs, on utilise la fonction `SUM()`.

In [ ]:
SELECT SUM (Level) FROM players

## 2. Visualisation avec XVEGA_PLOT

Le principal atout du noyau `xsqlite` est sa capacité à générer des graphiques à partir de requêtes SQL en utilisant la grammaire de visualisation [Vega-Lite](https://vega.github.io/vega-lite/).

La commande magique `%XVEGA_PLOT` transforme le résultat d'une requête `SELECT` en un graphique.

### Premier graphique : un nuage de points

Visualisons la relation entre le niveau (`Level`) et les points de vie (`Hitpoints`) de nos personnages.

````{admonition} Syntaxe de XVEGA_PLOT
:class: note
La structure de la commande est la suivante :
`%XVEGA_PLOT [OPTIONS] <> REQUETE_SQL`
Où les options de base sont :
- `X_FIELD`: Le nom de la colonne pour l'axe des X.
- `Y_FIELD`: Le nom de la colonne pour l'axe des Y.
- `MARK`: Le type de marqueur visuel à utiliser (`circle`, `bar`, `line`, `square`, `area`...).
- `WIDTH` / `HEIGHT`: Les dimensions du graphique en pixels.
````

In [ ]:
%XVEGA_PLOT X_FIELD Level Y_FIELD Hitpoints MARK circle WIDTH 100 HEIGHT 200 <> SELECT Level, Hitpoints FROM players

### Explorer d'autres options de visualisation

Pour montrer plus de fonctionnalités, chargeons une base de données plus complète, `chinook.db`, qui contient des données sur des pistes musicales, des artistes et des employés d'un magasin de musique.

Nous utilisons pour cela la commande magique `%LOAD`.

In [ ]:
%LOAD chinook.db

#### Regroupement de données (`BIN`)

L'option `BIN TRUE` permet de regrouper les données en "bacs" ou intervalles, ce qui est utile pour créer des histogrammes.

In [ ]:
%XVEGA_PLOT X_FIELD EmployeeId Y_FIELD ReportsTo BIN TRUE MARK square WIDTH 100 HEIGHT 200 <> SELECT EmployeeId, ReportsTo FROM employees

#### Personnalisation des bacs et des couleurs

On peut spécifier le nombre maximal de bacs avec `MAXBINS` et changer la couleur avec `COLOR`.

In [ ]:
%XVEGA_PLOT X_FIELD EmployeeId Y_FIELD ReportsTo BIN MAXBINS 3 MARK bar COLOR red WIDTH 200 HEIGHT 200 <> SELECT EmployeeId, ReportsTo FROM employees

#### Spécifier le type de données (`TYPE`)

L'option `TYPE` permet de préciser la nature des données (`quantitative`, `nominal`, `ordinal`, `temporal`). Ici, `ordinal` indique que les données ont un ordre intrinsèque.

In [ ]:
%XVEGA_PLOT X_FIELD EmployeeId Y_FIELD ReportsTo TYPE ordinal MARK bar COLOR green WIDTH 200 HEIGHT 200 <> SELECT EmployeeId, ReportsTo FROM employees

#### Graphique en ligne (`line`)

Le marqueur `line` est idéal pour visualiser des tendances. Utilisons la clause `LIMIT` en SQL pour ne sélectionner que les 10 premiers artistes.

In [ ]:
%XVEGA_PLOT X_FIELD Name TYPE nominal Y_FIELD ArtistId BIN MAXBINS 1 MARK line COLOR purple WIDTH 200 HEIGHT 200 <> SELECT Name, ArtistId FROM artists LIMIT 10

#### Graphique en aires (`area`) et options d'affichage

Le marqueur `area` remplit l'espace sous une ligne. L'option `GRID false` permet de masquer la grille de fond du graphique.

In [ ]:
%XVEGA_PLOT X_FIELD EmployeeId GRID false Y_FIELD ReportsTo MARK area COLOR pink WIDTH 200 HEIGHT 200 <> SELECT EmployeeId, ReportsTo FROM employees

## 3. Autres commandes magiques utiles

Le noyau `xsqlite` fournit d'autres commandes pour gérer votre environnement.

### Vérifier l'existence d'une table
La commande `%TABLE_EXISTS` renvoie `True` ou `False` selon que la table spécifiée existe ou non dans la base de données courante.
Rechargeons notre première base de données pour tester.

In [ ]:
%LOAD example_db.db

In [ ]:
%TABLE_EXISTS players

In [ ]:
%TABLE_EXISTS npcs

### Obtenir des informations sur la connexion
La commande `%GET_INFO` affiche des détails sur la base de données actuellement chargée.

In [ ]:
%GET_INFO

### Supprimer la base de données

La commande `%DELETE` ferme la connexion et supprime le fichier de la base de données du disque.

````{admonition} Attention
:class: warning
Cette opération est irréversible ! Toutes les données contenues dans le fichier `example_db.db` seront perdues.
````

In [ ]:
%DELETE

## Conclusion

Félicitations ! Vous avez parcouru les bases de SQL pour créer, remplir et interroger une base de données. Vous avez également découvert comment utiliser la puissance de `XVEGA_PLOT` pour créer des visualisations de données directement dans votre notebook Jupyter.

N'hésitez pas à expérimenter avec d'autres types de requêtes, d'autres options de visualisation et d'autres ensembles de données pour approfondir vos connaissances.